# Multi Objective Multi Agent Pathfinding Subject to Vehicle Models

## Overview
- loading packages
- Performing single runs
- Visualizing single runs
- Visualising multiple runs from the DB
- Running multiple experiments and saving to DB

## Objectives
- Makespan: Number of steps of the longest path
- Flowtime: Mean number of steps for all agents
- Robustness:
  * Positive: Shortest distance 
    * of an agent to other agents
    * half the distance to the wall
    * reasoning: an agent has radius r and the bigger r could be the better, min distance between two agents is twice agents to wall
  * Negative: In case an agent crosses through an obstacle fraction of the infeasible steps

In [ ]:
%pip install --upgrade pip 
%pip install --upgrade numpy dubins deap matplotlib pandas ipympl seaborn ipywidgets sqlalchemy gitpython nbstripout pre-commit

In [ ]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc, animation
import itertools
from IPython.display import display
import pandas as pd
import seaborn as sns
import cProfile
import pstats

import ipywidgets as widgets

from deap import base, creator, tools, algorithms

rc("animation", html="jshtml")

from path import *
from obstacle_map import *
from problem import *
from experiment import *

import sqlalchemy

engine = sqlalchemy.create_engine('sqlite:///experiments.db')

## Running the Algorithm

In [ ]:
settings = {
    'radius': 10, # turning radius (dubins vehicle)
    'model': Vehicle.DUBINS, # vehicle model
    'step': 1, # step size for simulated behaviour
    'domain': (0, 200.0), # area of operation (-100, 100) means that the vehicles will move in a square from (x=-100, y=-100) to (x=100, y=100)
    'n_agents': 5, # number of agents
    'n_waypoints': 3, # waypoints per agent (excluding start and end)
    'n_gens': 100, # number of generations to run the algorithm
    'population_size': 4*10, # population size for the algorithm, shoulod be divisible by 4 for nsga2
    'cxpb': 0.3, # crossover probablity
    'mutpb': 1.0, # mutation rate (not applicable in nsga2)
    'mutation_p': (1.0, 4.0, 5.0), # distribution of mutation types
    'sigma' : 0.2, # sigma for gauss-distribution in waypoint-gauss-mutation
    'feasiblity_threshold': 95, # how robust a solution has to be to be regarded feasible (100-min_dist)
    'offset': (0, 0), # offset of the map to the agents
    'map_name': "cross.obstacles.npy", # name of the obstacle-map file
    'metric': Metric.MIXED, # metric to use in fitness calculation
}

In [ ]:
profiling = False
experiment = Experiment(settings) # load the settings
experiment.setup() # setup population and deap-toolbox
experiment.seed(42)
if profiling:
    profile = cProfile.Profile()
    profile.enable()
pop, logbook = experiment.run() # start running :)
if profiling:
    profile.disable()

In [ ]:
if profiling:
    stats = pstats.Stats(profile)
    stats.sort_stats("tottime")
    stats.print_stats()

## Visualization of single runs

- plot general data
- plot best solutions
- animation for best solution (use filename="FOO.mp4" to save a video file)
- visualize mutation and crossover operators

In [ ]:
# select 5 best individuals (non-dominated sorting)
best = experiment.toolbox.select(pop, 5)

In [ ]:
for ind in best:
    print(ind.fitness.values)
    experiment.problem.solution_plot(ind, plot_range=range(0, 200))

In [ ]:
for i, sol in enumerate(best):
    experiment.problem.solution_animation(sol, plot_range=range(0,200))#, filename=f"with_obstancle_{i}.mp4")

In [ ]:
sol = toolbox.individual()
problem.solution_plot(sol, plot_range=range(0, 200))
print(sol)
print(problem.encode(problem.decode(sol)))

problem.uniform_mutation(sol, debug = True)
print(sol)
problem.solution_plot(sol, plot_range=range(0, 200))
problem.mutate(sol)
problem.solution_plot(sol, plot_range=range(0, 200))
problem.skip_mutation(sol, debug=True)
problem.solution_plot(sol, plot_range=range(0, 200))
#problem.waypoints_to_path(problem.decode(sol))

In [ ]:
plt.close('all')

## Saving and Visualisation - Multiple runs with DB

* works with `sqlalchemy` package and `sqlite` in dev environment
* currently uses `experiments.db` saved to `engine` variable
* adding and removing jobs to the db
* running jobs
* visualisation

In [ ]:
settings = {
    'radius': 10,
    'step': 1,
    'domain': (0, 200.0),
    'n_agents': 5,
    'n_waypoints': 3,
    'n_gens': 500,
    'population_size': 4*25,
    'cxpb': 0.3,
    'mutpb': 1.0,
    'mutation_p': (1.0, 4.0, 5.0),
    'sigma' : 0.2,
    'model': Vehicle.DUBINS,
    'feasiblity_threshold': 95,
    'offset': (0, 0),
    'map_name': "cross.obstacles.npy",
    'metric': Metric.MIXED,
}

job_settings = {
    "delete" : True,
    "runs" : 31,
    "name" : "baseline",
    "user" : "basti",
    "db" : engine,
}
add_jobs_to_db(settings, **job_settings)
job_settings['delete'] = False

#job_settings['name'] = "straight"
#settings['model'] = Vehicle.STRAIGHT
#add_jobs_to_db(settings, **job_settings)
#settings['model'] = Vehicle.DUBINS
settings['metric'] = Metric.MIN
job_settings['name'] = "min_metric"
add_jobs_to_db(settings, **job_settings)

settings['metric'] = Metric.MEAN
job_settings['name'] = "mean_metric"
add_jobs_to_db(settings, **job_settings)
settings['metric'] = Metric.MIXED


In [ ]:
df_jobs = pd.read_sql_table("jobs", con=engine)
for status in range(3):
    print(f"status {status}: {len(df_jobs.loc[df_jobs.status == status])}")
df_jobs.head(10)

In [ ]:
runner = ExperimentRunner(engine)
running = True
while running:
    running = runner.fetch_and_execute()
    

In [ ]:
df_pop = pd.read_sql("populations", con=engine)
plt.figure()
sns.scatterplot(data=df_pop, x="robustness", y="flowtime", palette=None, hue="crowding_distance", style="non_dominated", size_order=[True, False], size="non_dominated")
plt.show()

In [ ]:

df_jobs = pd.read_sql_table("jobs", con=engine)
for status in range(3):
    print(f"status {status}: {len(df_jobs.loc[df_jobs.status == status])}")

    
def get_names(db):
    df_pop = pd.read_sql("populations", con=db)
    print(df_pop["experiment"].unique())
    
def read_experiment(db, name=None):
    df_pop = pd.read_sql("populations", con=db)
    df_stats = pd.read_sql("logbooks", con=db)
    if name is not None:
        return df_pop.loc[df_pop['experiment']==name], df_stats.loc[df_stats['experiment']==name]
    return df_pop, df_stats

def fetch_settings(df_jobs, job_index=None):
    assert(job_index is not None)
    row = df_jobs.loc[df_jobs.index == job_index]
    s = row.iloc[0]
    return pickle.loads(s["settings"])

def plot_indivdual(row, df_jobs=df_jobs, plot=True, animation=False, animation_file=None):
    settings = fetch_settings(df_jobs, job_index=row['job_index'])
    ex = Experiment(settings)
    ex.setup()
    ind = pickle.loads(row['value'])
    if plot:
        ex.problem.solution_plot(ind)
    if animation:
        ex.problem.solution_animation(ind, filename=animation_file)
    return settings, ex

df_pop, df_stats = read_experiment(engine)
plt.close('all')

In [ ]:
# plot one run makespan-flowtime trade-off with non dominated solutions highlighted
plt.figure()
sns.scatterplot(data=df_pop.loc[df_pop['experiment']=="baseline"], x="makespan", y="flowtime", hue="robustness", style="non_dominated", size_order=[True, False], size="non_dominated")
plt.show()

In [ ]:
# plot non dominted solutions for all runs
plt.figure()
sns.scatterplot(data=df_pop.loc[df_pop["non_dominated"]], x="robustness", y="flowtime", hue="experiment")#, palette="jet")
plt.show()

In [ ]:
# plot one run makespan-flowtime trade-off with non dominated solutions highlighted
plt.figure()
sns.scatterplot(data=df_pop.loc[df_pop["experiment"]=="baseline"], x="robustness", y="flowtime", hue="crowding_distance", palette="plasma_r", style="non_dominated", size_order=[True, False], size="non_dominated")
plt.show()

In [ ]:
df_non_dom = df_pop.loc[df_pop["experiment"]=="baseline"].sort_values("robustness", ascending=True)
for i, row in df_non_dom[:5].iterrows():
    display(row)
    plot_indivdual(row, df_jobs=df_jobs, plot=True, animation=True)


In [ ]:
import traceback
traceback.print_last()

In [ ]:
def logbook_to_df(logbook):
    data = []
    evals = 0
    for log in logbook:
        evals += log['evals']
        data_i = {
            "generation": log['gen'],
            "evals": evals,
        }
        for i, _ in enumerate(log['median']):
            data_i[f"f_{i}_median"] = log['median'][i]
            data_i[f"f_{i}_min"] = log['min'][i]
            data_i[f"f_{i}_max"] = log['max'][i]
        data.append(data_i)
    return pd.DataFrame(data)
    
df_log = logbook_to_df(logbook)
df_log

In [ ]:
plt.figure()
sns.lineplot(data=df_log, x="generation", y="f_0_median")
plt.show()
plt.figure()
sns.lineplot(data=df_log, x="generation", y="f_1_median")
plt.show()

In [ ]:
JobStatus

In [ ]:
for s in JobStatus:
    print(s)